In [48]:
# use ansible to let argocd setup platform (minio, postgres, mlflow)

In [4]:
export PATH=/work/.local/bin:$PATH
export PYTHONUSERBASE=/work/.local
export ANSIBLE_CONFIG=/work/MLSysEngOps-NYU25Spring/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles

In [3]:
ll

total 884
drwxrwsr-x 9 zl5573_nyu_edu users   4096 May  9 23:28 ./
drwxr-sr-x 7 zl5573_nyu_edu users   4096 May  9 23:21 ../
drwxrwsr-x 8 zl5573_nyu_edu users   4096 May  6 20:33 ansible/
drwxrwsr-x 9 zl5573_nyu_edu users   4096 May  9 18:37 .git/
-rw-rw-r-- 1 zl5573_nyu_edu users     43 May  8 22:22 .gitignore
-rw-rw-r-- 1 zl5573_nyu_edu users    122 May  6 15:25 .gitmodules
drwxrwsr-x 2 zl5573_nyu_edu users   4096 May  6 15:25 images/
drwxrwsr-x 2 zl5573_nyu_edu users   4096 May  9 23:18 .ipynb_checkpoints/
drwxrwsr-x 6 zl5573_nyu_edu users   4096 May  6 15:25 k8s/
-rw-rw-r-- 1 zl5573_nyu_edu users   1061 May  6 15:25 LICENSE
-rw-rw-r-- 1 zl5573_nyu_edu users   4922 May  6 20:33 LICENSE.txt
-rw-r--r-- 1 zl5573_nyu_edu users  60923 May  9 19:04 project_Iac_delete.ipynb
-rw-rw-r-- 1 zl5573_nyu_edu users 741803 May  9 21:01 project_IaC.ipynb
-rw-rw-r-- 1 zl5573_nyu_edu users   2100 May  6 20:33 project_key.ipynb
-rw-rw-r-- 1 zl5573_nyu_edu users  14313 May  9 23:28 project_stages.ipynb


In [6]:
cd /work/MLSysEngOps-NYU25Spring/workflows

In [7]:
cat build-initial.yaml

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: kaniko-pvc-build-
spec:
  entrypoint: kaniko-build-flow
  volumeClaimTemplates:
  - metadata:
      name: workdir
    spec:
      accessModes: [ "ReadWriteOnce" ]
      resources:
        requests:
          storage: 1Gi

  templates:
  - name: kaniko-build-flow
    steps:
      - - name: clone-repo
          template: git-clone
      - - name: kaniko-build
          template: kaniko

  - name: git-clone
    container:
      image: alpine/git
      command: [sh, -c]
      args:
        - |
          MAX_RETRIES=5
          i=1

          while [ $i -le $MAX_RETRIES ]; do
            echo "Attempt $i of $MAX_RETRIES: Cloning repo..."
            git clone --branch app https://github.com/zuchuandatou/MLSysEngOps-NYU25Spring.git /mnt/workspace && break
            echo "Git clone failed. Retrying in 5 seconds..."
            i=$((i + 1))
            sleep 5
          done

          if [ ! -d /mnt/workspace/.git ]; 

In [8]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_platform.yml


PLAY [Mount persistent volume and deploy MLflow platform with MinIO] ***********

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Detect filesystem on /dev/vdb] *******************************************
ok: [node1]

TASK [Format /dev/vdb as ext4 on first use] ************************************
skipping: [node1]

TASK [Make root mount directory] ***********************************************
ok: [node1]

TASK [Mount volume & ensure /etc/fstab] ****************************************
ok: [node1]

TASK [Make MinIO data dir] *****************************************************
ok: [node1]

TASK [Make Postgres data dir] **************************************************
ok: [node1]

PLAY [Deploy MLflow platform via ArgoCD & Helm with MinIO secret handling] *****

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password from Kubernetes secret] *********************

In [13]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/workflow_build_init.yml


PLAY [Run Argo Workflow from GitHub Repo] **************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Clone or update gourmetgram-iac repo] ************************************
changed: [node1]

PLAY [Run Argo Workflow from GitHub Repo] **************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Clone or update gourmetgram-iac repo] ************************************
ok: [node1]

TASK [Submit Argo Workflow] ****************************************************
changed: [node1]

TASK [Extract Workflow Name] ***************************************************
ok: [node1]

TASK [Wait for workflow to complete (success or fail)] *************************
changed: [node1]

TASK [Get final workflow result] ***********************************************
changed: [node1]

TASK [Display workflow phase] *********************************

: 2

In [ ]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_staging.yml

In [ ]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_canary.yml

In [ ]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_prod.yml

In [ ]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/workflow_templates_apply.yml